## Usage test of the wrapper


In [1]:
# imports
import pandas as pd
import numpy as np

import logging

# from wrapper import RustDataModule
from rust_time_series.rust_time_series import (
    ForecastingDataSet,
    ClassificationDataSet,
    DatasetType,
    ImputeStrategy,
    SplittingStrategy,
)

In [2]:
# set up logging
logging.basicConfig(level=logging.INFO)

In [25]:
# create dummy 2D array
dummy = np.random.randint(0, 100, (100, 10)).astype(float)

# Create a RustTimeSeries instance
dummy_ts = ForecastingDataSet(dummy, DatasetType.Forecasting, 3, 3, 1)
dummy_ts.set_to_100()


dummy

array([[100.,  73.,  75.,  23.,  55.,  69.,  27.,  64.,  79.,  59.],
       [ 75.,  34.,  82.,  87.,  17.,  74.,  47.,  52.,   8.,   9.],
       [ 45.,  86.,   9.,  31.,  35.,  11.,  77.,  81.,  22.,   7.],
       [ 54.,  16.,  12.,  20.,  87.,  64.,   8.,  26.,  69.,  42.],
       [ 42.,   9.,  89.,  21.,  80.,  69.,  45.,  70.,  14.,  99.],
       [ 42.,  73.,  50.,  55.,  68.,  69.,  76.,  24.,  37.,  12.],
       [ 18.,  44.,  54.,  71.,  66.,  10.,  94.,  31.,  87.,  72.],
       [ 56.,  68.,   0.,  43.,  61.,  50.,  14.,  43.,  23.,  91.],
       [ 10.,  56.,  14.,  50.,  56.,  52.,  71.,   9.,  49.,  20.],
       [  3.,  80.,  45.,   7.,  67.,   3.,  91.,  75.,  79.,  14.],
       [ 89.,   7.,  33.,  19.,  48.,   2.,  74.,  18.,  88.,   6.],
       [ 71.,  41.,  80.,  83.,  47.,  42.,  80.,  24.,  93.,  12.],
       [ 34.,  77.,  46.,  80.,  94.,  41.,  51.,  70.,  40.,   4.],
       [ 16.,  89.,  37.,  13.,  78.,   7.,  80.,   0.,   0.,  19.],
       [ 61.,  95.,  88.,  74.,   

In [27]:
dummy_ts.get(1).sequence()

array([[75., 34., 82., 87., 17., 74., 47., 52.,  8.,  9.],
       [45., 86.,  9., 31., 35., 11., 77., 81., 22.,  7.],
       [54., 16., 12., 20., 87., 64.,  8., 26., 69., 42.]])

In [5]:
split = (0.7, 0.2, 0.1)
# confirm that split adds up to 1
# assert sum(split) == 1.0, "Split proportions must sum to 1."

(part1, part2, part3) = dummy_ts.split(SplittingStrategy.Temporal, *split)

print(len(part1), len(part2), len(part3))

70 20 10


### Dummy classification dataset

In [6]:
labels = np.ones((100, 1), dtype=np.float64)
labels = labels.astype(str)
labels_list = labels.flatten().tolist()

dummy_class = ClassificationDataSet(
    dummy, labels_list, DatasetType.Classification,
)

dummy_class.len()

100

### Forecasting Dataset

In [7]:
# Load the dataset
file_path = "LD2011_2014.txt"
df = pd.read_csv(file_path, sep=";", decimal=",")
# drop first column (date)
df = df.drop(columns=["date"])
# turn pandas DataFrame into numPy array
data = df.to_numpy(dtype=np.float64)

data

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.53807107e+00, 2.06258890e+01, 1.73761946e+00, ...,
        1.53589316e+02, 6.70087977e+02, 6.86486486e+03],
       [1.26903553e+00, 2.13371266e+01, 1.73761946e+00, ...,
        1.46911519e+02, 6.46627566e+02, 6.54054054e+03],
       [2.53807107e+00, 1.99146515e+01, 1.73761946e+00, ...,
        1.31886477e+02, 6.73020528e+02, 7.13513514e+03]])

In [8]:
ts = ForecastingDataSet(data, DatasetType.Forecasting, 3, 3, 3)
ts.set_to_100()

print(type(data))
print(data.shape)

<class 'numpy.ndarray'>
(140256, 370)


In [9]:
(part1, part2, part3) = ts.split(SplittingStrategy.Temporal, 0.7, 0.2, 0.1)

len(part1), len(part2), len(part3)

(98179, 28051, 14026)

### Classification Dataset

In [10]:
from aeon.datasets import load_classification
X, y = load_classification("GunPoint")
X = X.reshape(X.shape[0], -1)

print(type(X))
print(X.shape)

<class 'numpy.ndarray'>
(200, 150)


In [11]:
ts = ClassificationDataSet(X, y, DatasetType.Classification)

Usage of RustDataModule


In [12]:
from wrapper import RustDataModule

Rust Time Series Wrapper Loaded


In [13]:
rust_dm = RustDataModule(data, DatasetType.Forecasting, 3, 3, 3)

In [14]:
rust_dm.setup()

In [15]:
train_dl = rust_dm.train_dataloader()

train_dl

In [16]:
# test wether the dataloader works
for batch in train_dl:
    print(batch)
    print(batch.shape)
    break

tensor([[100.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.]], dtype=torch.float64)
torch.Size([32, 370])
